In [2]:
import sys
sys.path.append('/home/kmouraviev')

In [3]:
from tensorflow_resnet.my_batch_normalization import MyBatchNormalization

In [5]:
import tensorflow as tf

In [6]:
def berhu_loss(labels, predictions, delta=1.0):
    error = tf.abs(labels - predictions)
    loss = tf.where(error < delta,
                    error,
                    (error ** 2 + delta ** 2) / (2 * delta)
                   )
    return tf.reduce_mean(loss)

In [7]:
from keras.models import load_model
model = load_model('model_on_epoch17.hdf5', custom_objects={'MyBatchNormalization': MyBatchNormalization,
                                                            'berhu_loss': berhu_loss})

In [10]:
import h5py
import numpy as np
val_data_file = h5py.File('/home/kmouraviev/NYU_dataset_hdf5/validation_data_different_scenes.hdf5', 'r')
rgbs_val = np.array(val_data_file['data'])
depths_val = np.array(val_data_file['label'])

In [11]:
val_predictions = model.predict(rgbs_val, batch_size=23)

In [12]:
(val_predictions < 0.5).mean()

0.001405624759727017

In [17]:
print('Absolute error:', np.mean(np.abs(val_predictions[depths_val > 0.5] - \
                                        depths_val[depths_val > 0.5]) / \
                                 depths_val[depths_val > 0.5]))
print('RMSE:', np.sqrt(np.mean((val_predictions[depths_val > 0.5] - depths_val[depths_val > 0.5]) ** 2)))

Absolute error: 0.18907362368374672
RMSE: 0.6803062092492703


In [18]:
our_predictions = val_predictions[depths_val > 0.5]
our_gts = depths_val[depths_val > 0.5]
print('RMS log:', np.sqrt(np.mean((np.log(our_predictions) - np.log(our_gts)) ** 2)))

RMS log: 0.25720508607294706


In [19]:
rel_error = np.maximum(our_predictions / our_gts, our_gts / our_predictions)
print(rel_error.min(), rel_error.max())

1.0000000000053673 64.98293430754191


In [20]:
delta_1 = np.mean(rel_error < 1.25)
delta_2 = np.mean(rel_error < 1.25 ** 2)
delta_3 = np.mean(rel_error < 1.25 ** 3)

In [21]:
print('delta:', delta_1)
print('delta_2:', delta_2)
print('delta_3:', delta_3)

delta: 0.721933524906959
delta_2: 0.926548367452541
delta_3: 0.9802851989520613
